In [16]:
from dqn.dqn_base import ConcreteDQN
from dqn.neural_net import NeuralNetworkWithCNN
from dqn.memory import ExperienceReplay
import gymnasium as gym
import torch
import random
import matplotlib.pyplot as plt
import numpy as np
import IPython
from tensorboard import notebook

In [2]:
def preprocess_state(state, device):
    # print(state.dtype)
    state = np.expand_dims(state, axis=0)
    # print("Process state", state.shape)
    # state = state.transpose((2, 0, 1))
    # print(state.shape)
    state = torch.from_numpy(state)
    state = state.to(device, dtype=torch.float32)
    state = state.unsqueeze(1)
    # state = state.transpose((2, 0, 1))
    return state

In [3]:
device = torch.device("cpu")
# device = torch.device("mps")

env_id = "PongNoFrameskip-v4"
# env_id = "ALE/Pong-v5"
# env = make_atari(env_id)
# env = wrap_deepmind(env, frame_stack=True)

# env = gym.make(env_id, render_mode="human")
env = gym.make(env_id)
# env = gym.wrappers.AtariPreprocessing(env, frame_skip=4)
env = gym.wrappers.AtariPreprocessing(env)

seed_value = 23

torch.manual_seed(seed_value)
random.seed(seed_value)

learning_rate = 0.0001
num_episodes = 500
gamma = 0.99

hidden_layer = 512

batch_size = 32
experience_memory_size = 100000

update_target_frequency = 2000

report_interval = 20
number_of_inputs = env.observation_space.shape[0]
number_of_outputs = env.action_space.n


A.L.E: Arcade Learning Environment (version 0.8.0+919230b)
[Powered by Stella]


In [4]:
qnet_agent = ConcreteDQN(
    env,
    NeuralNetworkWithCNN(number_of_inputs, hidden_layer, number_of_outputs).to(device),
    learning_rate,
    device,
    gamma,
    preprocess_state_func=preprocess_state,
    egreedy=0.01,
    egreedy_final=0.01,
    egreedy_decay=10000,
    save_model_frequency=10000,
    resume_previous_training=True,
    experience_memory_size=experience_memory_size,
    batch_size=batch_size,
    target_net=NeuralNetworkWithCNN(number_of_inputs, hidden_layer, number_of_outputs).to(device),
    update_target_frequency=update_target_frequency
)

Loading model
Loading model


In [22]:
qnet_agent.learn(num_episodes, seed_value, report_interval)

0 21.0 0.01
1 18.0 0.01
Saving model
2 16.0 0.01
3 21.0 0.01
4 13.0 0.01
5 18.0 0.01
Saving model
6 17.0 0.01
